## Clustering of single-cell RNA-Seq PBMC data using the Galapagos workflow (working example)

````
Author: Florian Wagner <florian.wagner@uchicago.edu>
Date: 9/15/2019
````

This notebook contains an example of how to apply the Galapagos clustering workflow to single-cell RNA-Seq data in Python. Its execution requires the packages `numpy`, `pandas`, `scikit-learn`, and `plotly`. The code was run with Python 3.7, numpy 1.16.2, pandas 0.24.2, scikit-learn 0.20.3, and plotly 3.7.0. The dataset analyzed is called "10k PBMCs from a Healthy Donor (v3 chemistry)" and is available for [download](https://support.10xgenomics.com/single-cell-gene-expression/datasets/3.0.0/pbmc_10k_v3) from the 10x Genomics website.

This notebook uses a preprocessed version of the PBMC dataset (```pbmc_v3_10k_expression.npz```) that is included in the GitHub repository alongside this notebook. The preprocessing workflow consisted of 1) filtering for protein-coding genes, 2) mapping gene identifiers from Ensembl IDs to names, 3) filtering for cells that meet basic QC criteria. Preprocessing does not change the expression measurements (UMI counts) contained in the raw data.

A detailed description of Galapagos, as well as the preprocessing workflow applied to the raw data, can be found in the [Galapagos paper on bioRxiv (Wagner, 2019)](https://www.biorxiv.org/content/10.1101/770388v1). 

### Configure the Jupyter notebook

In [1]:
# change notebook width and font
from IPython.core.display import display, HTML
display(HTML("""<style>
    /* source: http://stackoverflow.com/a/24207353 */
    .container { width:95% !important; }    
    div.prompt, div.CodeMirror pre, div.output_area pre { font-family:'Hack', monospace; font-size: 10.5pt; }
    </style>"""))

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# for use without internet connection, use "connected = False"
# (this will add 5 MB to the size of the notebook!)
connected = True

# initialize the plotly plotting library
init_notebook_mode(connected=connected)

### Read expression data

In [2]:
import os
import pandas as pd
import numpy as np

expression_file = 'data/pbmc_v3_10k_expression.npz'

def load_binary_expression_file(file_path: str) -> pd.DataFrame:
    """Loads expression data stored in binary numpy format."""
    file_path_expanded = os.path.expanduser(file_path)
    file_size_mb = os.path.getsize(file_path_expanded) / 1e6

    data = np.load(file_path_expanded)
    genes = data['genes']
    cells = data['cells']
    X = np.array(data['matrix'].T, order='F', copy=False)
    matrix = pd.DataFrame(index=genes, columns=cells, data=X)

    print('Loaded %d x %d expression matrix -- '
          '.npz format, %.1f MB, path: %s'
          % (matrix.shape[0], matrix.shape[1], file_size_mb, file_path))
    return matrix

matrix = load_binary_expression_file(expression_file)
print('The shape of the expression matrix is:', matrix.shape)

Loaded 16319 x 10392 expression matrix -- .npz format, 35.7 MB, path: data/pbmc_v3_10k_expression.npz
The shape of the expression matrix is: (16319, 10392)


### Normalize and transform the data

In [3]:
# normalize to the median and transform using the Freeman-Tukey transformation

def normalize(matrix: pd.DataFrame) -> pd.DataFrame:
    """Normalize each profile to the median total transcript count."""
    num_transcripts = matrix.sum(axis=0)
    transcript_count = num_transcripts.median()
    print('The median transcript count is: %.1f' % transcript_count)
    norm_matrix = (transcript_count / num_transcripts) * matrix
    return norm_matrix


def stabilize_variance(matrix: pd.DataFrame) -> pd.DataFrame:
    """Apply the Freeman-Tukey transformation."""
    return np.sqrt(matrix) + np.sqrt(matrix + 1)


trans_matrix = stabilize_variance(normalize(matrix))

The median transcript count is: 5831.0


### Perform PCA

In [4]:
from sklearn.decomposition import PCA

num_components = 30

# scikit-learn can perform a truncated PCA
# using the randomized PCA algorithm by Halko et al.,
# which is much faster than computing
# all principal components
pca_model = PCA(
    n_components=num_components,
    svd_solver='randomized',
    random_state=0)

pc_scores = pca_model.fit_transform(trans_matrix.T).T

print('The first %d PCs explain %.1f %% of the total variance.'
      % (num_components, 100*pca_model.explained_variance_ratio_.sum()))
print('The shape of the PC score matrix is: %s' % (str(pc_scores.shape)))

The first 30 PCs explain 34.2 % of the total variance.
The shape of the PC score matrix is: (30, 10392)


### Perform t-SNE (on PC scores)

In [5]:
from sklearn.manifold import TSNE

perplexity = 30
seed = 0

# scikit-learn uses the Barnes-Hut algorithm for t-SNE
tsne_model = TSNE(perplexity=perplexity, random_state=seed)
Y = tsne_model.fit_transform(pc_scores.T).T

# convert numpy array to pandas DataFrame
dim_labels = ['tSNE dim. 1', 'tSNE dim. 2']
tsne_scores = pd.DataFrame(data=Y, index=dim_labels, columns=matrix.columns)

print('The shape of the t-SNE score matrix is: %s' % (str(tsne_scores.shape)))

The shape of the t-SNE score matrix is: (2, 10392)


### Plot the t-SNE result (using plotly)

In [6]:
trace = go.Scatter(
    x=tsne_scores.iloc[0],
    y=tsne_scores.iloc[1],
    mode='markers',
    marker=dict(size=5, opacity=0.7, color='navy'),
)

data = [trace]

layout = go.Layout(
    width=820,
    height=800,
    font=dict(size=32, family='serif'),
    xaxis=dict(title='tSNE dim. 1', zeroline=False, showline=True, tickvals=[], showgrid=False),
    yaxis=dict(title='tSNE dim. 2', zeroline=False, showline=True, tickvals=[], showgrid=False),
    margin=dict(l=60, b=60),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Apply DBSCAN to the t-SNE result

In [7]:
from sklearn.cluster import DBSCAN

eps_frac = 0.0260
minPts_frac = 0.007


def cluster_cells_dbscan(
        tsne_scores: pd.DataFrame,
        eps_frac: float = 0.03,
        minPts_frac: float = 0.01) -> pd.Series:
    """Cluster cells using DBSCAN."""

    # convert parameters
    ptp = np.ptp(tsne_scores.values, axis=1)
    eps = eps_frac * pow(np.sum(np.power(ptp, 2.0)), 0.5)
    minPts = minPts_frac * tsne_scores.shape[1]
    print('eps: %.2f' % eps)
    print('minPts: %d' % minPts)

    # perform DBSCAN
    model = DBSCAN(algorithm='brute', eps=eps, min_samples=minPts)
    y = model.fit_predict(tsne_scores.T)
    
    # convert numpy array to pandas Series
    cell_labels = pd.Series(index=matrix.columns, data=y)
    
    # fix cluster labels
    vc = cell_labels.value_counts()
    cluster_labels = dict([
        (id_, 'Cluster %d' % (i+1)) for i, id_ in enumerate(vc.index)])
    cluster_labels[-1] = 'Outliers'
    cell_labels = cell_labels.map(cluster_labels)
    
    return cell_labels


cell_labels = cluster_cells_dbscan(
    tsne_scores, eps_frac, minPts_frac)

# print cluster sizes
print(cell_labels.value_counts())

eps: 5.45
minPts: 72
Cluster 1     3320
Cluster 2     1938
Cluster 3     1346
Cluster 4      901
Cluster 5      898
Cluster 6      566
Cluster 7      443
Outliers       307
Cluster 9      249
Cluster 10     156
Cluster 11     113
Cluster 12      80
Cluster 13      75
dtype: int64


### Plot the clustering results (using plotly)

In [8]:
cluster_colors = {
    'Outliers': 'gray',
}

def plot_clustering_results(
        tsne_matrix: pd.DataFrame,
        cell_labels: pd.Series,
        cluster_colors: dict = None):
    
    if cluster_colors is None:
        cluster_colors = {}
    
    # determine plotting order
    cluster_order = []
    value_counts = cell_labels.value_counts()
    cluster_number = 1
    for id_ in value_counts.index:
        if id_ == 'Outliers':
            continue
        cluster_order.append('Cluster %d' % cluster_number)
        cluster_number +=1    
    if 'Outliers' in value_counts.index:
        cluster_order.append('Outliers')

    # plot all clusters
    data = []
    for cluster_name in cluster_order:

        color = None
        try:
            color = cluster_colors[cluster_name]
        except KeyError:
            pass
        
        sel_cells = (cell_labels == cluster_name)
        
        trace = go.Scatter(
            x=tsne_scores.iloc[0].loc[sel_cells],
            y=tsne_scores.iloc[1].loc[sel_cells],
            mode='markers',
            marker=dict(size=5, opacity=0.7, color=color),
            name=cluster_name,
        )
        
        data.append(trace)
        
    layout = go.Layout(
        width=1000,
        height=800,
        font=dict(size=32, family='serif'),
        xaxis=dict(title='tSNE dim. 1', zeroline=False, showline=True, tickvals=[], showgrid=False),
        yaxis=dict(title='tSNE dim. 2', zeroline=False, showline=True, tickvals=[], showgrid=False),
        margin=dict(l=60, b=60),
        legend=dict(x=1.00, xanchor='left', y=0.98, yanchor='top', bordercolor='black', borderwidth=1),
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)    

    
plot_clustering_results(tsne_scores, cell_labels, cluster_colors=cluster_colors)

## Copyright and License

Copyright (c) 2019 Florian Wagner.

This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

The code in this notebook is also licensed under the following license.

```
BSD 3-Clause License

Copyright (c) 2019, Florian Wagner

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```